In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import os

In [2]:
import sys
from pathlib import Path

# Add project root (parent of the optimization folder) to sys.path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [3]:
THEATER_NAME = "AMC Boston Common 19"
NUM_SCREENS = 19
SCREENS = [f"Screen_{i}" for i in range(1, NUM_SCREENS+1)]

In [4]:
# Days and screens
DAYS = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
OPENING_TIME = dt.time(11,0)
CLOSING_TIME=dt.time(23,0)

today = dt.date.today()
open_dt = dt.datetime.combine(today, OPENING_TIME)
close_dt = dt.datetime.combine(today, CLOSING_TIME)
delta = close_dt - open_dt


# Operating hours per screen per day (e.g., 10 hours = 600 minutes)
OPERATING_MIN_PER_DAY = {
    day: delta.total_seconds()/60 for day in DAYS  # same for all days for now
}

# Theater capacities per screen
SCREEN_CAPACITIES = {s: 210 
for s in SCREENS[:19]}
SCREEN_CAPACITIES["Screen_19"] = 600

# Buffer time between shows (e.g., cleaning, trailers, turnover) in minutes
BUFFER_MIN = 15

# Max number of showings per movie per day (frequency constraint)
MAX_SHOWINGS_PER_MOVIE_PER_DAY = 5

# Synthetic movie Data 
movies_df = pd.DataFrame({
    "movie_id": ["M1", "M2", "M3", "M4"],
    "title": ["Blockbuster Action", "Indie Drama", "Family Animation", "Horror Thriller"],
    "runtime_min": [130, 110, 95, 100],
    "ticket_price": [15.0, 14.0, 13.0, 13.0]
})

In [5]:
from tmdb_api_calling.utils import get_movies_out_now, get_movies_from_url

In [6]:
tmdb_key = os.getenv("TMDB_API_KEY")

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {tmdb_key}"
}

UPCOMING_MOVIES_URL = "https://api.themoviedb.org/3/movie/upcoming"

In [7]:
CURRENT_MOVIES = get_movies_out_now(headers)

In [ ]:
MOVIES_OUT_NOW_DF = pd.DataFrame(CURRENT_MOVIES)

In [24]:
MOVIES_OUT_NOW_DF = MOVIES_OUT_NOW_DF[(pd.to_datetime(MOVIES_OUT_NOW_DF.release_date)>pd.to_datetime("2025-11-01"))
&
(MOVIES_OUT_NOW_DF.original_language=="en")
]

In [25]:
MOVIES_OUT_NOW_DF.drop_duplicates(subset="original_title", inplace=True)

/var/folders/mq/p952ryqs1gv79cnvz9f05slr0000gn/T/ipykernel_33257/3159563086.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MOVIES_OUT_NOW_DF.drop_duplicates(subset="original_title", inplace=True)


In [26]:
MOVIES_OUT_NOW_DF

,id,imdb_id,title,original_title,original_language,release_date,status,homepage,budget,revenue,...,origin_country,spoken_languages,genre_ids,genre_names,production_company_ids,production_company_names,production_company_countries,production_country_codes,production_country_names,belongs_to_collection
3,1248226,tt31425731,Playdate,Playdate,en,2025-11-05,Released,https://www.amazon.com/gp/video/detail/B0FQ1MCFKJ,0,0,...,US,English,"28, 35, 10751","Action, Comedy, Family","110723, 28787, 221397, 215935, 55446, 376","Nickel City Pictures, Wide Awake Pictures, A H...","US, US, US, US, CA, US","US, CA","United States of America, Canada",None
4,1242898,tt31227572,Predator: Badlands,Predator: Badlands,en,2025-11-05,Released,https://www.20thcenturystudios.com/movies/pred...,105000000,136304860,...,US,English,"28, 878, 12","Action, Science Fiction, Adventure","127928, 840, 1302, 185309, 22213","20th Century Studios, Lawrence Gordon Producti...","US, US, US, US, US",US,United States of America,Predator Collection
7,425274,tt4712810,Now You See Me: Now You Don't,Now You See Me: Now You Don't,en,2025-11-12,Released,https://nowyouseeme.movie,90000000,80654261,...,US,English,"53, 80, 9648","Thriller, Crime, Mystery","90517, 1632, 85205, 114336, 491, 279807","Secret Hideout, Lionsgate, Epic Films, Reese W...","US, US, AU, US, US, US","US, AU","United States of America, Australia",Now You See Me Collection
9,967941,tt19847976,Wicked: For Good,Wicked: For Good,en,2025-11-16,Released,https://www.wickedmovie.com,150000000,0,...,US,English,"10749, 14, 12","Romance, Fantasy, Adventure","33, 2527","Universal Pictures, Marc Platt Productions","US, US",US,United States of America,Wicked Collection
12,1117857,tt27604215,In Your Dreams,In Your Dreams,en,2025-11-07,Released,https://www.netflix.com/title/80992977,0,0,...,US,English,"16, 10751, 35, 12, 14","Animation, Family, Comedy, Adventure, Fantasy","144322, 171251","Kuku Studios, Netflix Animation Studios","US, US",US,United States of America,None
14,1084242,tt26443597,Zootopia 2,Zootopia 2,en,2025-11-26,Released,,0,0,...,US,English,"16, 10751, 35, 12, 9648","Animation, Family, Comedy, Adventure, Mystery",6125,Walt Disney Animation Studios,US,US,United States of America,Zootopia Collection
17,1425122,tt35600079,A Very Jonas Christmas Movie,A Very Jonas Christmas Movie,en,2025-11-10,Released,https://www.disneyplus.com/browse/entity-0a35b...,0,0,...,US,English,"35, 10402","Comedy, Music","236166, 274572, 152686, 12292, 208750, 280547","20th Television, Copper Cup Entertainment, The...","US, US, US, US, US, US",US,United States of America,None
18,798645,tt14107334,The Running Man,The Running Man,en,2025-11-11,Released,https://www.runningmanmovie.com,110000000,28200000,...,US,English,"28, 53, 878","Action, Thriller, Science Fiction","4, 131464, 28788, 216687","Paramount Pictures, Complete Fiction, Genre Fi...","US, GB, US, US","US, GB","United States of America, United Kingdom",None


# Optimization Formulation

1. Parameters

- $A_i,d$: expected attendance for movie i per day d

- $P_d$: average ticket price for movie on day d
- $D_i$: duration of movie i in minutes
- $B$: buffer time between shows
- $C_j$: capacity of screen j
- $H_j,d$: hours screen j operates on day d 
- $K$: maximum times a movie can be shown on a given day


2. Decision Variables:

- $s_{i,d}$: { 0, 1 }: whether movie i is scheduled on day d
- $x_{i, d, j}$: number of times we show movie i on screen j on a given day d 
- $r_{i, d}$: realized tickets served for movie i on day d, where $r_i,d=\min(j  x_i, d,j C_j,  A_i,d) $

3. Objective function: Maximize daily revenue:

$$max (i, d,j\times P_d r_i,d   ) $$


4. Constraints
- Total screen time: $\sum_{j} x_{i,d,j} (D_i+B) \leq H_{d,j} \forall d,j$
- Tickets served won’t exceed demand:  $r_i,d\leq \sum_{j} x_i, d,j C_j$
- Can’t sell more tickets than available: $r_i,d \leq A_i,d$  
- If not scheduled, no showings:  $\sum_{j} x_i, d,j \leq M*s_i,d$
- Each movie shown at most K times: $\sum_{j} x_i, d,j \leq K  ∀ i,d$

In [ ]:
from audioop import maxpp


def createModel(runtimes, demand, ticket_price, max_showings_per_day, operating_min_per_day, screen_capacity, movie_ids, days, screens, buffer_min, verbose=False):

    m = gp.Model("CinemaShowtimeScheduling")
    if not verbose:
        m.Params.OutputFlag = 0  # silence output by default
    
    x = m.addVars(
        movie_ids, days, screens,
        vtype=GRB.INTEGER,
        name="x"
    )
    
    # r[i,d] >= 0 continuous
    r = m.addVars(
        movie_ids, days,
        vtype=GRB.CONTINUOUS,
        lb=0.0,
        name="r"
    )
    
    # s[i,d] binary
    s = m.addVars(
        movie_ids, days,
        vtype=GRB.BINARY,
        name="s"
    )
    
    # Objective: maximize total revenue
    m.setObjective(
        gp.quicksum(ticket_price[i] * r[i, d] for i in movie_ids for d in days),
        GRB.MAXIMIZE
    )
    
    # 1) Screen time constraint per screen & day
    #   sum_i x[i,d,j] * (runtime_i + buffer) <= operating_min_per_day[d]
    for d in days:
        for j in screens:
            m.addConstr(
                sum(x[i, d, j] * (runtimes[i] + buffer_min) for i in movie_ids)
                <= operating_min_per_day[d],
                name=f"Time_{d}_{j}"
            )
    
    # 2) Capacity: r[i,d] <= sum_j x[i,d,j] * capacity_j
    for i in movie_ids:
        for d in days:
            m.addConstr(
                r[i, d] <= gp.quicksum(x[i, d, j] * screen_capacity[j] for j in screens),
                name=f"Capacity_{i}_{d}"
            )
    
    # 3) Demand: r[i,d] <= A[i,d]
    for i in movie_ids:
        for d in days:
            demand_id = demand.get((i, d), 0)
            m.addConstr(
                r[i, d] <= demand_id,
                name=f"Demand_{i}_{d}"
            )
    
    # 4) Frequency + schedule linking
    for i in movie_ids:
        for d in days:
            # Frequency limit: total showings per day
            # Link to s: if s[i,d] = 0 => no showings
            m.addConstr(
                sum(x[i, d, j] for j in screens)
                <= max_showings_per_day * s[i, d],
                name=f"FreqLimit_{i}_{d}"
            )

    
         
    return m
    

In [ ]:
model = None

In [ ]:
model.optimize()

In [ ]:
def optimize_showtimes(model, verbose = False):

    model.optimize()

    status_code = m.Status
    if status_code == GRB.OPTIMAL:
        status = "Optimal"
    elif status_code == GRB.INFEASIBLE:
        status = "Infeasible"
    elif status_code == GRB.UNBOUNDED:
        status = "Unbounded"
    else:
        status = f"Status_{status_code}"
    
    total_revenue = m.objVal if status_code == GRB.OPTIMAL else None
    
    if verbose:
        print("Status:", status)
        print("Total revenue:", total_revenue)
    
    schedule_rows = []
    if status_code == GRB.OPTIMAL:
        for i in movie_ids:
            for d in days:
                for j in screens:
                    val = x[i, d, j].X
                    if val > 1e-6:
                        schedule_rows.append({
                            "movie_id": i,
                            "day": d,
                            "screen": j,
                            "showings": int(round(val))
                        })
    
    schedule_df = pd.DataFrame(schedule_rows)
    
    realized_rows = []
    if status_code == GRB.OPTIMAL:
        for i in movie_ids:
            for d in days:
                realized_rows.append({
                    "movie_id": i,
                    "day": d,
                    "realized_tickets": r[i, d].X,
                    "scheduled_flag": int(round(s[i, d].X))
                })
    realized_df = pd.DataFrame(realized_rows)
    
    return {
        "status": status,
        "total_revenue": total_revenue,
        "schedule_df": schedule_df,
        "realized_df": realized_df
    }
    